# **Load Dataset**

jika dijalankan di gdrive import drive nya dulu lalu sesuaikan path nya

In [ ]:
# from google.colab import drive
# drive.mount ('/content/drive')

: 

# SCRAPING

In [ ]:
%pip install Sastrawi
%pip install google-play-scraper

: 

In [ ]:
from google_play_scraper import app
import pandas as pd
import numpy as np

: 

In [ ]:
from google_play_scraper import Sort, reviews

result, continuation_token = reviews(
    'nozero.apps1', # ID = google play
    lang='id', 
    country='id',
    sort=Sort.MOST_RELEVANT, 
    count=200, 
    filter_score_with= 5  # defaults to None(means all score) Use 1 or 2 or 3 or 4 or 5 to select certain score
)

: 

In [ ]:
# Dataframe dengan nama
df_ulasan = pd.DataFrame(np.array(result),columns=['review'])
df_ulasan = df_ulasan.join(pd.DataFrame(df_ulasan.pop('review').tolist()))
df_ulasan.head()

: 

In [ ]:
df_ulasan[['content', 'score']].head()

: 

In [ ]:
my_ulasan = df_ulasan[['content', 'score']]

: 

In [ ]:
my_ulasan.to_csv("riliv-rating5.csv", index = False)

: 

# Penggabungan Data .CSV

In [ ]:
%ls

: 

In [ ]:
df_rating5 = pd.read_csv("riliv-rating5.csv")
df_rating3 = pd.read_csv("riliv-rating3.csv")
df_rating1 = pd.read_csv("riliv-rating1.csv")


: 

In [ ]:
df_ratingAll = pd.concat([df_rating5,df_rating3,df_rating1])

: 

In [ ]:
len(df_ratingAll)

: 

In [ ]:
df_ratingAll.to_csv("ratingAll.csv", index = False)

: 

# Preprocessing

In [ ]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


# Load the dataset
dframe = pd.read_csv("ratingAll.csv")

: 

## **Case Folding**

In [ ]:
# Case Folding
data_casefolded = dframe['content'].str.lower()

: 

## **Filtering**

In [ ]:
# Filtering
def filter_text(text):
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)  # Remove URLs
    text = re.sub(r'\(cont\)', ' ', text)  # Remove continuations
    text = re.sub(r'[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', text)  # Remove punctuation
    text = re.sub(r'#\S+', '', text)  # Remove hashtags
    text = re.sub(r'\d+', ' ', text)  # Remove numbers
    return text

data_filtered = data_casefolded.apply(filter_text)

: 

## **Tokenization**

In [ ]:
# Tokenization
nltk.download('punkt')
data_tokenized = data_filtered.apply(word_tokenize)

print(data_tokenized)

: 

## **Stopword Removal**

In [ ]:
# Stopword Removal
factory = StopWordRemoverFactory()
stopword_list = factory.get_stop_words()

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopword_list and len(word) > 3]

data_no_stopwords = data_tokenized.apply(remove_stopwords)

print(data_no_stopwords)

: 

## **Slang Word Normalization**

In [ ]:
path_dataslang = open("kamus.csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

ulasan_formal = []
for data in data_no_stopwords:
  data_clean = [replaceSlang(word) for word in data]
  ulasan_formal.append(data_clean)
len_ulasan_formal = len(ulasan_formal)
print(ulasan_formal)
len_ulasan_formal

: 

## **Stemming**

In [ ]:
factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
white_list = ["bali"]

def stemmer(line):
  valid = list()
  for word in line:
    if(word not in white_list):
      word = ind_stemmer.stem(word)
    if(len(word)>2):
      valid.append(word)
  return valid

data_stemmed = [stemmer (line) for line in ulasan_formal]
print(data_stemmed)

: 

In [ ]:
unique_words = []
for word in ulasan_formal:
    if word not in unique_words:
        unique_words.append(word)

# Menggabungkan kembali kata-kata unik menjadi teks
unique_text = unique_words

print(unique_text)

: 

# **Join**

In [ ]:
# Join tokens back to string
data_preprocessed = [' '.join(tokens) for tokens in data_stemmed]
print(data_preprocessed)
#data_steammed berfungsi untuk mengubah data yang awalan array perkata menjadi satu kalimat

: 

# **Feature Extraction**

In [ ]:
# Feature Extraction with TFIdf
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_preprocessed)
y = dframe['score']

: 

# **Modelling**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

: 

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)  # tentukan jumlah tetangga (K)
knn.fit(X_train, y_train)

# Prediksi data uji
knn_y_pred = knn.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, knn_y_pred)
print(f'Akurasi model KNN: {accuracy:.2f}')

report_knn = classification_report(y_test, knn_y_pred)
print ('report : ' +str(report_knn))

: 

## SVM

In [ ]:
from sklearn import svm

# Inisialisasi dan latih model SVM
svm = svm.SVC(kernel='linear')
svm.fit(X_train, y_train)

# Prediksi data uji
y_pred = svm.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi model SVM: {accuracy:.2f}')

report = classification_report(y_test, y_pred)
print ('report : ' +str(report))

: 

# Saving Model

Disini model terbaik menggunakan SVM jadi menggunakan model SVM dengan tingkat akurasi 0.78

In [ ]:
import pickle

# Save the model
with open('svm_model.pkl', 'wb') as file:
    pickle.dump(svm, file)

print("Model saved successfully.")

: 

: 